In [ ]:
!pip install ta
!pip install yahoo-finance
!pip install pandas
!pip install numpy
!pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read Dataset
df = pd.read_excel('C:/Users/yunus/Downloads/Data Deneme/Dataset.xlsx')

In [ ]:
# Check Data columns
print(df.head())

In [ ]:
# Dimensions of the dataset
print(df.shape)  

In [ ]:
# Display basic information about the dataset
print(df.info())

In [ ]:
# Mining Hash Rate type is object, it should be float64
df['Mining Hash Rate'].unique()
df['Mining Hash Rate'].isna().sum()
df['Mining Hash Rate'] = pd.to_numeric(df['Mining Hash Rate'], errors='coerce')
df['Mining Hash Rate'].dtype

In [ ]:
# Check for duplicates
print(df.duplicated().sum())

In [ ]:
# Check for constant columns
constant_columns = [col for col in df.columns if df[col].std() == 0]
print(f'Constant columns: {constant_columns}')

# Drop constant columns if any
# df = df.drop(columns=constant_columns)

In [ ]:
# Converting Date column to index
df['DateTime'] = df.index
df.set_index("DateTime", inplace=True)


In [ ]:
# Check again after indexing date
print(df.info())

In [ ]:
# Summary statistics
summary_stats = df.describe()
summary_stats

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
missing_values

In [ ]:
total_missing = df.isna().sum().sum()
total_missing

In [ ]:
# Visualize missing values with an expanded heatmap
plt.figure(figsize=(40, 20))  # Increase figure size to accommodate more columns
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Values Heatmap')
plt.xticks(rotation=90)  # Rotate column labels for better visibility
plt.show()

In [ ]:
# Handling Missing Values
# Forward fill missing values for all columns
df = df.ffill()
# there are still missing values after forward filling, backward filling fills them with the next valid observation
df = df.bfill()

In [ ]:
# Visualize missing values with an expanded heatmap
plt.figure(figsize=(40, 20))  # Increase figure size to accommodate more columns
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Values Heatmap')
plt.xticks(rotation=90)  # Rotate column labels for better visibility
plt.show()

In [ ]:
# Histograms
df.hist(figsize=(30, 25), bins=30, edgecolor='black')
plt.suptitle('Histograms of Numerical Variables')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
# Kernel density estimate (KDE) plots for each numerical variable
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    sns.kdeplot(df[column], fill=True)
    plt.title(f'Distribution of {column}')
    plt.show()

In [ ]:
# Calculate skewness and kurtosis for each feature
from scipy.stats import gaussian_kde, skew, kurtosis

# Select only numerical columns
# numerical_df = df.select_dtypes(include=['float64', 'int64']) # if there is any non-numerical variable in data

skewness = df.apply(lambda x: skew(x.dropna()))
kurt = df.apply(lambda x: kurtosis(x.dropna()))

# Plot Skewness
plt.figure(figsize=(70, 20))
plt.subplot(2, 1, 1)
skewness.plot(kind='bar', color='skyblue')
plt.title('Skewness')
plt.xlabel('Features')
plt.ylabel('Skewness')
plt.xticks(rotation=90)

# Plot Kurtosis
plt.subplot(2, 1, 2)
kurt.plot(kind='bar', color='lightcoral')
plt.title('Kurtosis')
plt.xlabel('Features')
plt.ylabel('Kurtosis')
plt.xticks(rotation=90)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
# Identify potential outliers in numerical features
# Plot boxplots for numerical features
plt.figure(figsize=(50, 50))
for i, column in enumerate(df.select_dtypes(include=['float64', 'int64']).columns, 1):
    plt.subplot(15, 15, i)  # Adjust the number of rows and columns based on data
    sns.boxplot(df[column])
    plt.title(column)
plt.tight_layout()
plt.show()

In [ ]:
# Scatter Plots for Bivariate analysis
target_variable = 'BTC_Adj Close'

# Get all other variables except the target
other_variables = [col for col in df.columns if col != target_variable]

# Create scatter plots for the target variable against each other variable
for variable in other_variables:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=df[variable], y=df[target_variable])
    plt.title(f'Scatter Plot of {variable} vs {target_variable}')
    plt.xlabel(variable)
    plt.ylabel(target_variable)
    plt.show()

In [ ]:
# Correlation matrix
correlation_matrix = df.corr()

In [ ]:
# Visualize the correlation matrix
plt.figure(figsize=(40, 40))  # Adjust the figure size
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', cbar=True)
plt.title('Correlation Matrix')
plt.xticks(rotation=90)  # Rotate labels for better visibility
plt.yticks(rotation=0)  # Keep y-axis labels horizontal
plt.show()

In [ ]:
# Heatmap for Correlation with Target Variable
# Extract the correlation values of the target variable 'Adj Close'
target_correlation = correlation_matrix['BTC_Adj Close'].sort_values(ascending=False)

# Reorder the correlation matrix based on the target variable correlation
reordered_corr_matrix = df[target_correlation.index].corr()

# Mask to show only the lower triangle
mask = np.triu(np.ones_like(reordered_corr_matrix, dtype=bool))

# Plot the heatmap with improved aesthetics and without annotation
plt.figure(figsize=(40, 40))
sns.heatmap(
    reordered_corr_matrix,
    annot=False,  # Remove the numbers
    cmap='coolwarm',
    fmt=".2f",
    linewidths=0.5,
    linecolor='black',
    cbar_kws={'shrink': 0.8},
    square=True,
    mask=mask  # Apply the mask to hide the upper triangle
)

plt.title('Correlation Heatmap with Target Variable: BTC_Adj Close', fontsize=16, pad=20)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.show()



In [ ]:
# Stationarity Checks
# Unit Root Test
# Function to perform the Augmented Dickey-Fuller test
from statsmodels.tsa.stattools import adfuller


def adf_test(series, name):
    result = adfuller(series.dropna())
    labels = ['ADF Test Statistic', 'p-value', '# Lags Used', '# Observations Used']
    out = pd.Series(result[0:4], index=labels)
    
    for key, value in result[4].items():
        out['Critical Value (%s)' % key] = value
    
    print(f'Results of Augmented Dickey-Fuller Test for {name}:')
    print(out)
    print('Stationary' if result[1] <= 0.05 else 'Non-Stationary')
    print('---' * 20)

# List of columns to check for stationarity (including the target variable)
columns_to_check = ['BTC_Adj Close', 'ETH_Close', 'BTC_Close_lag1', 'BTC_Close_lag2', 'BTC_Close_lag3', 
                    'BTC_Close_lag4', 'BTC_Close_lag5', 'BTC_Close_lag10', 'BTC_Close_lag20', 
                    'BTC_Close_lag30', 'BTC_MA5', 'BTC_MA7', 'BTC_MA10', 'BTC_MA14', 'BTC_MA20', 
                    'BTC_MA30', 'BTC_MA50', 'BTC_MA100', 'BTC_MA200', "Petroleum Markets EMV Tracker"]

# Perform the ADF test on each of the selected columns
for column in columns_to_check:
    adf_test(df[column], column)

In [ ]:
# Stationarity Test
# Function to perform the KPSS test
from statsmodels.tsa.stattools import kpss

def kpss_test(series, name):
    result = kpss(series.dropna(), regression='c')
    labels = ['KPSS Test Statistic', 'p-value', '# Lags Used']
    out = pd.Series(result[0:3], index=labels)
    
    for key, value in result[3].items():
        out['Critical Value (%s)' % key] = value
    
    print(f'Results of KPSS Test for {name}:')
    print(out)
    print('Non-Stationary' if result[1] <= 0.05 else 'Stationary')
    print('---' * 20)

# List of columns to check for stationarity (including the target variable)
columns_to_check = ['BTC_Adj Close', 'ETH_Close', 'BTC_Close_lag1', 'BTC_Close_lag2', 'BTC_Close_lag3', 
                    'BTC_Close_lag4', 'BTC_Close_lag5', 'BTC_Close_lag10', 'BTC_Close_lag20', 
                    'BTC_Close_lag30', 'BTC_MA5', 'BTC_MA7', 'BTC_MA10', 'BTC_MA14', 'BTC_MA20', 
                    'BTC_MA30', 'BTC_MA50', 'BTC_MA100', 'BTC_MA200', "Petroleum Markets EMV Tracker"]

# Perform the KPSS test on each of the selected columns
for column in columns_to_check:
    kpss_test(df[column], column)

In [ ]:
# Stationarity Visualization
from statsmodels.tsa.seasonal import seasonal_decompose

# Decompose the series
decomposition = seasonal_decompose(df['BTC_Adj Close'].dropna(), model='additive', period=40) # adjust the period based on data's seasonality

# Extract the components
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Plot the decomposed components
plt.figure(figsize=(12, 8))

plt.subplot(411)
plt.plot(df['BTC_Adj Close'], label='Original Series')
plt.legend(loc='upper left')
plt.title('Original Series')

plt.subplot(412)
plt.plot(trend, label='Trend', color='orange')
plt.legend(loc='upper left')
plt.title('Trend Component')

plt.subplot(413)
plt.plot(seasonal, label='Seasonal', color='green')
plt.legend(loc='upper left')
plt.title('Seasonal Component')

plt.subplot(414)
plt.plot(residual, label='Residual', color='red')
plt.legend(loc='upper left')
plt.title('Residual Component')

plt.tight_layout()
plt.show()

In [ ]:
# Autocorrelation plots
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# df is our DataFrame and 'Adj Close' is our time series column
time_series = df['BTC_Adj Close'].dropna()

# Plot Autocorrelation
plt.figure(figsize=(14, 7))
plot_acf(time_series, lags=40, ax=plt.gca())  # Adjust the 'lags' parameter as needed
plt.title('Autocorrelation Plot')
plt.show()

# Plot Partial Autocorrelation
plt.figure(figsize=(14, 7))
plot_pacf(time_series, lags=40, ax=plt.gca())  # Adjust the 'lags' parameter as needed
plt.title('Partial Autocorrelation Plot')
plt.show()



In [ ]:
# Cointegration Test
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# Johansen test has limits, 12 variables
# Selected 12 variables according to correlation matrix
selected_columns = ["BTC_Adj Close", "OBV", "TR", "Volatility",
                    "MSCI_World_Index", "Gold", "Federal Funds Effective Rate", "US_Dollar_Index", "S&P_500", 
                    "Discussion about pandemics, Index-Total count of mentions of epidemics (normalized by total number of words)", 
                    "Petroleum Markets EMV Tracker", "Financial Regulation EMV Tracker"] 
subset_df = df[selected_columns].dropna()  # Drop rows with NaN values

# Perform the Johansen test (assuming df contains multiple time series)
johansen_test = coint_johansen(subset_df, det_order=0, k_ar_diff=1)

# Extract the test statistics and critical values
trace_stat = johansen_test.lr1
max_eigen_stat = johansen_test.lr2
critical_values = johansen_test.cvt

print(f'Trace Statistics: {trace_stat}')
print(f'Max-Eigen Statistics: {max_eigen_stat}')
print(f'Critical Values (90%, 95%, 99%): \n{critical_values}')

# Check for cointegration based on the Trace statistic and critical values
print("\nCointegration Check based on Trace Statistic:")
for i in range(len(trace_stat)):
    if trace_stat[i] > critical_values[i, 1]:  # 95% level
        print(f"Series are cointegrated at rank {i+1}")
    else:
        print(f"No cointegration at rank {i+1}")
        break

# Check for cointegration based on the Max-Eigen statistic and critical values
print("\nCointegration Check based on Max-Eigen Statistic:")
for i in range(len(max_eigen_stat)):
    if max_eigen_stat[i] > critical_values[i, 1]:  # 95% level
        print(f"Series are cointegrated at rank {i+1}")
    else:
        print(f"No cointegration at rank {i+1}")
        break


In [ ]:
# Save the DataFrame to an Excel file
df.to_excel('Custom-Dataset.xlsx', index=True)